<a href="https://colab.research.google.com/github/dlasdowsky/Python/blob/main/Exerc%C3%ADcio_final_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício avaliativo de Python Basic


In [ ]:
!pip install ping3
!sudo apt install nmap
!pip install python3-nmap
import csv
import datetime
import smtplib
from email.mime.text import MIMEText
import socket
import ping3
import nmap3 
import os
from datetime import datetime
from ping3 import ping


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
nmap is already the newest version (7.80+dfsg1-2build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Crie uma classe enviarMensagens que tenha como variável padrão para inicialização a mensagem (Deve estar no __init__)


*   a classe deve ter um método enviar email com parâmetros email e titulo e estes serão usados na biblioteca de envio de email
*   Use a biblioteca smtplib





In [ ]:
import smtplib

class EnviarMensagens:
    def __init__(self, mensagem):
        self.mensagem = mensagem
        
    def enviar_email(self, destinatario, assunto):
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.ehlo()
            smtp.starttls()
            smtp.ehlo()
            #digitar o email do remetente e senha para logn
            smtp.login('seu_email@gmail.com', 'sua_senha')
            corpo_email = f'Subject: {assunto}\n\n{self.mensagem}'
            smtp.sendmail('seu_email@gmail.com', destinatario, corpo_email)

2. Crie uma classe com os seguintes métodos:
 

*   verificar_servidor (Use o socket) como parâmetro, o método usa o ip/dominio e a porta, deve salvar em um arquivo csv dia e hora e o resultado ( se conseguiu ou não), por fim, deve enviar email caso tenha dado errado.
*   verificar_portas (Use o nmap) como parâmetro, insira o dominio para teste e o resultado deve ser salvo em um arquivo csv para analises
*   realizar_ping (use uma lib de ping de sua preferência) como parâmetro entre o ip/domínio e salve o resultado em um arquivo csv, caso tenha erro no ping, envie email informando o erro, o dia e hora do ocorrido.


---


**ps.: os dados não podem ser apagados e sim inseridos ao final do arquivo**





In [ ]:


class VerificadorServidores:

    def __init__(self,ip_dominio):
        self.ip_dominio = ip_dominio        

    def verificar_portas(self):
        resultado = []
        mensagem = ""
        try:
            nmap = nmap3.Nmap()
            nm = nmap.scan_top_ports(self.ip_dominio)            
            i=0
            while i< len(nm[self.ip_dominio]['ports']):                            
              resultado.append(nm[self.ip_dominio]['ports'][i]['portid'])           
              i+=1     
            id_porta = 0    
            for i in range(len(resultado)):
                porta = resultado[i]                
                mensagem = self.verificar_servidor(porta)
        except Exception as e:
            mensagem = 'Erro ao verificar portas'
            #self.enviar_email(resultado, "0")
        finally:
            self.salvar_resultado('verificar_portas',  self.ip_dominio, "0", resultado)
        return mensagem;

    def verificar_servidor(self, porta):
        mensagem = ''
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(5)
            s.connect((self.ip_dominio, int(porta)))
            resultado = 'OK'
        except Exception as e:
            mensagem = 'Erro ao verificar servidor/porta'
            #self.enviar_email(resultado, porta)
        finally:
            s.close()
            self.salvar_resultado('verificar_servidor', self.ip_dominio, porta, resultado)
        return mensagem;

    def realizar_ping(self):
        resultado = ''
        try:
            rtt = ping(self.ip_dominio)            
            if rtt is not None:
                resultado = f'Ping bem-sucedido: {rtt:.3f} ms'
            else:
                resultado = 'Erro ao realizar o ping'
                self.enviar_email(resultado, self.ip_dominio)
        except Exception as e:
            resultado = f'Erro ao realizar o ping: {str(e)}'
            self.enviar_email(resultado, self.ip_dominio)
        finally:
            self.salvar_resultado('realizar_ping', self.ip_dominio, '', resultado)

    def salvar_resultado(self, metodo, ip_dominio, porta, resultado):
        
        # obter data e hora atual
        data_hora_atual = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # verificar se o arquivo CSV já existe
        arquivo_existente = True
        try:
            with open("Resultados.csv", 'r') as f:
                pass
        except FileNotFoundError:
            arquivo_existente = False
        # abrir arquivo CSV em modo de escrita, adicionando dados no final do arquivo caso já exista
        with open("Resultados.csv", 'a', newline='') as f:
            writer = csv.writer(f)
            if not arquivo_existente:
                writer.writerow(["data_hora", "metodo", "ip_dominio", "porta" , "resultado"])
            writer.writerow([data_hora_atual, metodo, ip_dominio, porta, resultado])
    


Rode o programa, verificando o servidor, as portas abertas e vendo o ping com o seguintes IP´s/DOminos
dominios_ips = ['186.202.153.153', 'www.google.com.br', 'minhacasa.edu.br']

In [ ]:
# Rode aqui seu código
 
servidor = VerificadorServidores( 'www.google.com.br')
destinatario = "dlasdowsky@gmail.com"
assunto = "Erro na validação do servidor"
VerificadorServidores.realizar_ping(servidor)
mensagem = (VerificadorServidores.verificar_portas(servidor))
if mensagem != "":
  enviarMensagens = EnviarMensagens(mensagem)
  EnviarMensagens.enviar_email(EnviarMensagens, destinatario, assunto)


0.0012733936309814453


SMTPAuthenticationError: ignored